In [1]:
from pandas import DataFrame
from sklearn.utils import shuffle
import os
import concurrent
import requests
import pandas as pd
from nbdev.showdoc import *
from fastcore.test import *
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot

import json

In [3]:
api_key = "785475a69770b7dc1afQ964feff948dd7"

In [4]:
data = pd.read_csv('train.csv')

In [5]:
valid = pd.read_csv('valid.csv')

In [6]:
def drop_col(data: DataFrame)->DataFrame:
        
        df = data.drop(['belongs_to_collection','homepage','imdb_id','production_companies','popularity','original_language','original_title','revenue','runtime','spoken_languages','status','video','vote_average','vote_count','release_date','production_countries','budget'],axis = 1)
        df =df.drop(df.columns[0], axis=1)
        
        return df

In [7]:
def clean_genre(df:DataFrame)-> DataFrame:
    
    mlb = MultiLabelBinarizer(sparse_output=True)
    df['genres'] = [[x['name'] for x in eval(list_dict)] for list_dict in df['genres']]
    df1 = df.join(
                pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('genres')),
                index=df.index,
                columns=mlb.classes_))
    return df1

In [8]:
def extract_backdrop_img(df:DataFrame):
    
    try:
        os.makedirs(".\\backdrop_img")
    except FileExistsError:
    # directory already exists
        pass
    
    for cnt,x in enumerate(df['backdrop_path']):
        
        if (x!= null):
            response = requests.get("https://image.tmdb.org/t/p/original"+str(x))
            file = open(".\\backdrop_img\\"+str(df['id'][cnt])+".jpg", "wb")
            file.write(response.content)
            file.close()
    
    return
    

In [9]:
def extract_poster_img(df:DataFrame):
    
    try:
        os.makedirs(".\\poster_img")
    except FileExistsError:
    # directory already exists
        pass
    
    for cnt,x in enumerate(df['poster_path']):
        
        if (x != None):
            response = requests.get("https://image.tmdb.org/t/p/original"+str(x))
            file = open(".\\poster_img\\"+str(df['id'][cnt])+".jpg", "wb")
            file.write(response.content)
            file.close()
    
    return
  

In [10]:
df = drop_col(data)

In [11]:
df1 = clean_genre(df)

In [12]:
df1.isnull().sum()

adult               0
backdrop_path      15
id                  0
overview            0
poster_path         0
tagline             0
title               0
Action              0
Adventure           0
Animation           0
Comedy              0
Crime               0
Documentary         0
Drama               0
Family              0
Fantasy             0
History             0
Horror              0
Music               0
Mystery             0
Romance             0
Science Fiction     0
TV Movie            0
Thriller            0
War                 0
Western             0
dtype: int64

In [18]:
extract_poster_img(df1)

NameError: name 'null' is not defined